## Preparing the 20 Newsgroups Dataset

We will need some data to work with. For the purposes of this demo we will make use of the 20 newsgroups dataset. Even though 20 newsgroups is a toy dataset, it offers enough complications to show how we can piece together embeddings using ``vectorizers``.

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
# from src import paths
# from src.data import Dataset

In [39]:
import numpy as np
import matplotlib.colors
import seaborn as sns
from sklearn.utils import Bunch

In [8]:
from sklearn.datasets import fetch_20newsgroups
opts={"subset":"all", "remove":"('headers', 'footers', 'quotes')"}
ds_in = fetch_20newsgroups(**opts)

In [20]:
# ds_in = Dataset.load('20_newsgroups')
print(ds_in.DESCR[0:1000])

.. _20newsgroups_dataset:

The 20 newsgroups text dataset
------------------------------

The 20 newsgroups dataset comprises around 18000 newsgroups posts on
20 topics split in two subsets: one for training (or development)
and the other one for testing (or for performance evaluation). The split
between the train and test set is based upon a messages posted before
and after a specific date.

This module contains two loaders. The first one,
:func:`sklearn.datasets.fetch_20newsgroups`,
returns a list of the raw texts that can be fed to text feature
extractors such as :class:`~sklearn.feature_extraction.text.CountVectorizer`
with custom parameters so as to extract feature vectors.
The second one, :func:`sklearn.datasets.fetch_20newsgroups_vectorized`,
returns ready-to-use features, i.e., it is not necessary to use a feature
extractor.

**Data Set Characteristics:**

    =================   ==========
    Classes                     20
    Samples total            18846
    Dimensionality

First, we will do a little pruning: the 20 newsgroups dataset contains some extremely short posts (once the headers, footers and quotes are removed) that will muddy up the results. We will prune out any post less than 200 characters (200 is chosen somewhat arbitrarily).

In [10]:
prune_limit = 200

In [11]:
long_enough = [len(t) > prune_limit for t in ds_in.data]
targets = np.array(ds_in.target)[long_enough]
news_data = [t for t in ds_in.data if len(t) > prune_limit]

In [43]:
ds = Bunch(data = new_data, 
           filenames = ds_in.filenames, 
           target_names = ds_in.target_names, 
           target = targets,
          DESCR = ds_in.DESCR)

For each newsgroup post, the target data is the name of the newsgroup the post was sent to. There are broad groups that the specific newsgroups can be classified into, such as religion, politics, computing, sport and science. While some of broad groups can be gleaned programmatically from the newsgroup name (with its dotted hierarchy), some groups (like alt.atheism being in the religion topic) require more care. We will hand curate the newsgroups into 6 broad categories:
* religion
* politics
* sport
* comp
* sci
* misc

Using these broad categories, we will create a custom color palette for the data when visualizing results such that different newsgroups in the same category can have similar colours.

In [44]:
religion = ("alt.atheism", "talk.religion.misc", "soc.religion.christian")
politics = ("talk.politics.misc", "talk.politics.mideast", "talk.politics.guns")
sport = ("rec.sport.baseball", "rec.sport.hockey")
comp = (
    "comp.graphics",
    "comp.os.ms-windows.misc",
    "comp.sys.ibm.pc.hardware",
    "comp.sys.mac.hardware",
    "comp.windows.x",
)
sci = (
    "sci.crypt",
    "sci.electronics",
    "sci.med",
    "sci.space",
)
misc = (
    "misc.forsale",
    "rec.autos",
    "rec.motorcycles",
)

In [45]:
color_key = {}
for l, c in zip(religion, sns.color_palette("Blues", 4)[1:]):
    color_key[l] = matplotlib.colors.rgb2hex(c)
for l, c in zip(politics, sns.color_palette("Purples", 4)[1:]):
    color_key[l] = matplotlib.colors.rgb2hex(c)
for l, c in zip(comp, sns.color_palette("YlOrRd", 5)):
    color_key[l] = matplotlib.colors.rgb2hex(c)
for l, c in zip(sci, sns.color_palette("light:teal", 5)[1:]):
    color_key[l] = matplotlib.colors.rgb2hex(c)
for l, c in zip(sport, sns.color_palette("light:#660033", 4)[1:3]):
    color_key[l] = matplotlib.colors.rgb2hex(c)
for l, c in zip(misc, sns.color_palette("YlGn", 4)[1:]):
    color_key[l] = matplotlib.colors.rgb2hex(c)
color_key["word"] = "#777777bb"

With a dataset and a carefully designed color palette we are in good shape to do some analysis of different embedding methods using UMAP to obtain visualizations of the embeddings. 

## Save this Dataset
Let's save this as a dataset for easy re-use in our other notebooks, and add the color palette to the metadata of the dataset. 

Note: This Dataset has already been added to the catalog and the following cells do not need to be run again. They are included here as a reference.

In [14]:
from src.helpers import notebook_as_transformer

ModuleNotFoundError: No module named 'src'

In [19]:
# new_dataset_name = f'{ds_in.name}_pruned'
new_data = news_data
new_target = targets
# new_metadata = ds_in.metadata.copy()
# new_metadata['color_key'] = color_key
# added_descr_txt = f"""\n This dataset is a subselection of the {ds_in.name} Dataset where we have pruned out any post less than {prune_limit} \
# characters ({prune_limit} is chosen somewhat arbitrarily). A custom `color_key` can be found in the metadata."""
# new_metadata['descr'] += added_descr_txt

new_ds = Dataset(dataset_name=new_dataset_name, data=new_data, target=new_target,
                 metadata=new_metadata)


NameError: name 'Dataset' is not defined

In [11]:
# Due to various design choiced in Jupyter, we need to specify this name manually.
nbname = '00-20-newsgroups-setup.ipynb'
dsdict = notebook_as_transformer(notebook_name=nbname,
                                 input_datasets=[ds_in],
                                 output_datasets=[new_ds],
                                 overwrite_catalog=True)

2023-03-30 14:06:53,120 - datasets - WARNING - Overwrite_catalog=True but generate=False. Not overwriting Dataset catalog entry for '20_newsgroups_pruned'
